In [2]:
#modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from collections import Counter
from geopy.geocoders import Nominatim
import re,  censusgeocode

In [101]:
# import 311 and clean
df311  = pd.read_csv('311data.csv',sep=',')
cols = ['Complete Address', 'Comments', 'Create Date', 'Department', 'Department ID', 'District', 'Master Request', 'Request ID', 'Request Type' , 'Latitude', 'Longitude' ]
df311 = df311[cols]

rqdf = pd.read_csv('requestTypes.csv')
rqdf = rqdf[[ 'Request Type', 'Request Category']]
transdict = {}
for i in range(rqdf.shape[0]):
    transdict[rqdf.loc[i][0]] = rqdf.loc[i][1]
df311['Request Type'] = df311['Request Type'].replace(transdict)
df311 = df311[df311['Request Type'] != 'x']
df311['Complete Address'] =  [i.lower() for i in df311['Complete Address']]
df311 = df311[df311['Complete Address'] != ', revere, ma']
df311.head()
#df311.to_csv('new311.csv') 

C:\Users\quinn\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (16,17,24,50,66,68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Complete Address,Comments,Create Date,Department,Department ID,District,Master Request,Request ID,Request Type,Latitude,Longitude
0,"57 & 59 eliot rd, revere, ma",Resident called to report unsanitary condition...,7/17/2017 20:16,ISD - Health,18,1,0,1025,Sanitation-ISD,42.401700,-70.990179
2,"6 harrington ave, revere, ma",Resident of Harrington Ave states that the pro...,7/18/2017 15:45,ISD - Health,18,5,0,1032,Sanitation-ISD,42.437756,-70.968747
3,"177 bellingham ave, revere, ma","Real estate agent called, Lenny Piazza (home o...",7/18/2017 20:38,ISD - Building,17,1,0,1038,Housing,42.396095,-70.985648
4,"1 case dr, revere, ma",Subcontractor left behind railroad ties and co...,7/19/2017 21:07,DPW,12,6,0,1052,Above Ground Infrastructure,42.419961,-71.022919
5,"mill st, revere, ma",A resident called the Mayor's office reporting...,7/20/2017 18:29,Economic Development,50,1,0,1063,Above Ground Infrastructure,42.404315,-71.011280


In [6]:
#import census and clean

censusdf = pd.read_csv('census.csv', header = 4)[['Address','Parcel ID', 'Census Block Group']]
unitre = re.compile(r' unit [\d|\s]*')
censusdf['Address'] = [i[:-7].lower() for i in censusdf['Address'] ]
censusdf['Address'] = [re.sub(unitre, '', i) for i in censusdf['Address'] ]
censusdf.head()

C:\Users\quinn\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (101,102,103,104,105,106,107,108) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Address,Parcel ID,Census Block Group
0,"36 lee burbank hwy, revere, ma",5-120A1-2A,County 025 Tract 981502 Block Group 1
1,"49 lee burbank hwy, revere, ma",5-80-4,County 025 Tract 981502 Block Group 1
2,"40 lee burbank hwy, revere, ma",5-120A1-2B,County 025 Tract 981502 Block Group 1
3,"251 lee burbank hwy, revere, ma",6-120B-15,County 025 Tract 981502 Block Group 1
4,"lee burbank hwy, revere, ma",5-120B-20,County 025 Tract 981502 Block Group 1


In [64]:
import time
cg = censusgeocode.CensusGeocode(benchmark='Public_AR_Current', vintage='Census2020_Current')
ret = []

for i in list(set(df311['Complete Address'])):
    time.sleep(.5)
    try:
        x = cg.onelineaddress(i)[0]['geographies']['Census Blocks'][0]
        ret +=[[i, x['BLOCK'], x['TRACT'] ]]
    except:
        print(i)
    


harrington st, revere, ma
baldwin st, revere, ma
riverside ave, revere, ma
420 ma-107, revere, ma
alger road, revere, ma
broadsound ave, revere, ma
100 lancaster ave, revere, ma
calumet st, revere, ma
east mountain ave, revere, ma
51 woodland rd, revere, ma
tapley ave, revere, ma
derby rd, revere, ma
282 ma-107, revere, ma
morris st, revere, ma
23 vet fgn wars park, revere, ma
prospect pl, revere, ma
124-154 , revere, ma
bixby st, revere, ma
260 clark rd, revere, ma
100 morris st, revere, ma
dolphin ave, revere, ma
south hancock st, revere, ma
vivien st, revere, ma
368 washington ave, revere, ma
joey rd, revere, ma
unity ave, revere, ma
211 squire rd, revere, ma
350 cecilian ave, revere, ma
taft st, revere, ma
camielle rd, revere, ma
oak tree ln, revere, ma
clark rd, revere, ma
123 bixby st, revere, ma
butler st, revere, ma
473 winthrop ave, revere, ma
fairfield st, revere, ma
gordon rd, revere, ma
9 rice ave, revere, ma
53 cushing ave, revere, ma
23 veterans of foreign wars pkwy, reve

In [85]:
insertDict = {}
for i in ret:
    insertDict[i[0]] = i[1:]

In [115]:
block, tract = [], []
for i in df311['Complete Address']:
    try:
        x = insertDict[i]
        block.append(x[0])
        tract.append(x[1])
    except:
        block.append(0)
        tract.append(0)

df311['Block'] = block
df311['Tract'] = tract


In [95]:
df311.to_csv('311census.csv')

In [125]:
#attempt 2, for addresses that were not recognized, using lat long isntead

addToCoord = {}
for i in range(df311.shape[0]):
    if df311.iloc[i]['Block'] == 0:

        if df311.iloc[i]['Complete Address'] not in addToCoord.keys():
            
            y = df311.iloc[i]['Latitude']
            x = df311.iloc[i]['Longitude']
            addToCoord[df311.iloc[i]['Complete Address']] = [x,y]

In [ ]:
ret2 = []
cg = censusgeocode.CensusGeocode(benchmark='Public_AR_Current', vintage='Census2020_Current')

for i in bad:
    cg.coordinates(x,y)

In [153]:
ret2 = []
bad2 = []
for i in list(addToCoord.keys()): 
    x, y = addToCoord[i]
    try:
        z = cg.coordinates(x,y)['Census Blocks'][0]
        ret2 += [[i,z['BLOCK'], z['TRACT'] ]]
    except:
        bad2 += [i]

print('Done [+]')

Done [+]


In [157]:
insertDict2 = {}
for i in ret2:
    '57 & 59 eliot rd, revere, ma'[i[0]] = i[1:]

In [177]:
for i in range(len(block)):
    z = (list(df311['Complete Address'])[i])
    try:
        x,y = (insertDict2[z])
        block[i] = x
        tract[i] = y
    except:
        pass
df311['Block'] = block
df311['Tract'] = tract
df311 = df311[df311['Block'] != 0]

In [186]:
df311.to_csv('311census.csv')

In [3]:
df311  = pd.read_csv('311census.csv',sep=',')
(Counter(((df311['Tract']))).most_common())

[(170601, 3571),
 (170702, 3110),
 (170301, 3033),
 (170800, 2693),
 (170400, 2356),
 (170200, 2025),
 (170102, 1878),
 (170502, 1407),
 (170101, 1379),
 (170701, 1032),
 (170302, 984),
 (170503, 789),
 (170504, 759),
 (981502, 83),
 (342401, 11),
 (210900, 3),
 (342102, 3),
 (207200, 2),
 (981600, 2),
 (341901, 1),
 (205600, 1),
 (354602, 1),
 (208102, 1),
 (160602, 1),
 (160501, 1),
 (754100, 1)]

In [12]:
ret = []
block = df311['Block']
tract = df311['Tract']
for i in range(len(block)):
    ret += [(tract[i],block[i])]


In [15]:
Counter(ret).most_common(10
)

[((170601, 2008), 434),
 ((170302, 1000), 274),
 ((170702, 5002), 272),
 ((170502, 2003), 269),
 ((170101, 2000), 267),
 ((170800, 2000), 265),
 ((170702, 3004), 238),
 ((170601, 1001), 237),
 ((170504, 1003), 236),
 ((170601, 2004), 217)]

In [ ]:
group='B02001'
data_url=f"https://api.census.gov/data/2018/acs/acs5/groups/{group}.json"
response=requests.get(data_url)
variable_labels_JSON=response.json()

with open('census_key.txt') as key:
    api_key=key.read().strip()

